In [1]:
def pars_group_old(group_id): 
    # В этот словарик будем собирать информацию о текстах, можно напихать что-нибудь еще
    wall_inf = {'text' : [ ],
               'id' : [ ]}
    

    # Выясним сколько всего в группе постов 
    n = my_api.wall.get(owner_id = '-' + group_id)['count']
    print('В группе ', n, 'постов')

    for i in tqdm_notebook(range(0, n + 100, 100)): 
        # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
        # Меняем время ожидания из идеи, что ВК отдает нам 3 запроса в секунду на токен
#         time.sleep(0.4)
        wall = my_api.wall.get(owner_id = '-' + group_id, count = 100, offset = str(i))['items']
        # Вытащили тексты постов и id
        new_text = [item['text'] for item in wall]
        post_ids = [item['id'] for item in wall]
        post_date = [item['date'] for item in wall]
        # Закинули всё это добро в словарик с данными 
        wall_inf['text'].extend(new_text)
        wall_inf['id'].extend(post_ids)
        wall_inf['date'].extend(post_date)
    return wall_inf


In [ ]:
def pars_group(group_id, api_array): 
    # В этот словарик будем собирать информацию о текстах, можно напихать что-нибудь еще
    wall_inf = {'text' : [ ],
               'id' : [ ],
               'date' : [ ]}
    
    # Это не многопоточность, но мне показалось, что нам надо вызывать не саму функцию несколько раз с разными
    # токенами в разных потоках, а внутри функции менять значения токенов. Отсюда родилась такая идея.
    # Делаем итератор из нашего массива, бесконечно гоняем его по кругу, уменьшаем время между запросами
    
    iter_api = itertools.cycle(api_array)
    len_api = len(api_array)
    my_api = next(iter_api)
    

    # Выясним сколько всего в группе постов 
    n = my_api.wall.get(owner_id = '-' + group_id)['count']
    print('В группе ', n, 'постов')

    for i in tqdm_notebook(range(0, n + 100, 100)): 
        # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
        # Меняем время ожидания из идеи, что ВК отдает нам 3 запроса в секунду на токен
        time.sleep(1/(3 * len_api))
#         my_api = next(iter_api)
        wall = my_api.wall.get(owner_id = '-' + group_id, count = 100, offset = str(i))['items']
        # Вытащили тексты постов и id
        new_text = [item['text'] for item in wall]
        post_ids = [item['id'] for item in wall]
        post_date = [item['date'] for item in wall]
        # Закинули всё это добро в словарик с данными 
        wall_inf['text'].extend(new_text)
        wall_inf['id'].extend(post_ids)
        wall_inf['date'].extend(post_date)
    return wall_inf


In [ ]:
# def pars_comment(pars_wall, api_array, group_id):    
#     comments_id = []
#     commentator_id = {}
    
#     iter_api = itertools.cycle(api_array)
#     len_api = len(api_array)
#     my_api = next(iter_api)

#     for post_id in tqdm_notebook(pars_wall['id']):
#         my_api = next(iter_api)
#         info = my_api.wall.getComments(owner_id= '-' + group_id, post_id=post_id, count = 100)['items']
#         topic = [
#             {            
#                 'comment_id': item['id'],
#                 'post_id' : post_id
#             } for item in info
#         ]
#         comments_id.extend(topic)

#         time.sleep(1/(3 * len_api))
        
#         dict_id_commenst = [
#             {
#                 'user_id': item['from_id']
#             } for item in info
#         ]

#         commentator_id[post_id] = dict_id_commenst
        
        
        
#     print('В группе ', len(comments_id), 'комментариев')
    
#     df_post_user = pd.DataFrame([(comment) for comment in commentator_id.items()], 
#                                 columns=['id', 'user_id_comment'])
#     df_post_user['user_id_comment'] = df_post_user['user_id_comment'].apply(lambda x: [id['user_id'] for id in x])
#     df_post_user.set_index('id', inplace=True)
    
#     return df_post_user, comments_id
    

In [ ]:
def pars_comment(pars_wall, api_array, group_id):    
    comments_id = []
    commentator_id = {}
    
    iter_api = itertools.cycle(api_array)
    len_api = len(api_array)
    

    for post_id in tqdm_notebook(pars_wall['id']):
        my_api = next(iter_api)
        n = my_api.wall.getComments(owner_id= '-' + group_id, post_id=post_id)['count']
        time.sleep(1/(3 * len_api))
        dict_id_commenst = []
        for i in range(0, n + 100, 100): 
            my_api = next(iter_api)
            info = my_api.wall.getComments(owner_id= '-' + group_id, 
                                           post_id=post_id, 
                                           count = 100, 
                                           offset = str(i))['items']
            topic = [
                {            
                    'comment_id': item['id'],
                    'post_id' : post_id
                } for item in info
                    ]
            comments_id.extend(topic)

            dict_id_commenst += [
                {
                    'user_id': item['from_id']
                } for item in info
                    ]
            
            time.sleep(1/(3 * len_api))
            
            
        commentator_id[post_id] = dict_id_commenst

        
    print('В группе ', len(comments_id), 'комментариев')
    
    df_post_user = pd.DataFrame([(comment) for comment in commentator_id.items()], 
                                columns=['id', 'user_id_comment'])
    df_post_user['user_id_comment'] = df_post_user['user_id_comment'].apply(lambda x: [id['user_id'] for id in x])
    df_post_user.set_index('id', inplace=True)
    
    return df_post_user, comments_id
    

In [ ]:
def pars_comment_old(pars_wall, group_id):    
    comments_id = []
    commentator_id = {}
    

    for post_id in tqdm_notebook(pars_wall['id']):
        info = my_api.wall.getComments(owner_id= '-' + group_id, post_id=post_id)['items']
        topic = [
            {            
                'comment_id': item['id'],
                'post_id' : post_id
            } for item in info
        ]
        comments_id.extend(topic)

        time.sleep(1/3)
        
        dict_id_commenst = [
            {
                'user_id': item['from_id']
            } for item in info
        ]

        commentator_id[post_id] = dict_id_commenst
        
        
        
    print('В группе ', len(comments_id), 'комментариев')
    
    df_post_user = pd.DataFrame([(comment) for comment in commentator_id.items()], 
                                columns=['id', 'user_id_comment'])
    df_post_user['user_id_comment'] = df_post_user['user_id_comment'].apply(lambda x: [id['user_id'] for id in x])
    df_post_user.set_index('id', inplace=True)
    
    return df_post_user, comments_id
    

In [2]:
# def pars_likes_users_post(pars_wall, api_array, group_id):    
#     likes_users_post = []
#     likes_id = {}
    
#     iter_api = itertools.cycle(api_array)
#     len_api = len(api_array)
#     my_api = next(iter_api)
    
#     for post_id in tqdm_notebook(pars_wall['id']):
#         my_api = next(iter_api)
#         try:
#             likes_id[post_id] = my_api.likes.getList(type='post',
#                                                      owner_id='-' + group_id,
#                                                      count = 1000,
#                                                      item_id=post_id)['items']
#         except:
#             likes_id[post_id] = []

#         time.sleep(1/(3 * len_api))
    
#     df_likes_user = pd.DataFrame([(like) for like in likes_id.items()], columns=['id', 'user_id_like'])
#     df_likes_user.set_index('id', inplace=True)
    
#     return df_likes_user

In [ ]:
def pars_likes_users_post(pars_wall, api_array, group_id):    
    likes_users_post = []
    likes_id = {}
    
    iter_api = itertools.cycle(api_array)
    len_api = len(api_array)
    
    for post_id in tqdm_notebook(pars_wall['id']):
        my_api = next(iter_api)
        try:
            n = my_api.likes.getList(type = 'post',
                                     owner_id= '-' + group_id, 
                                     item_id=post_id)['count']
        except:
            print('smth going wrong!!')
        time.sleep(1/(3 * len_api))
        tmp_like = []
        if n > 1000:
            for i in range(0, n + 1000, 1000): 
                try:
                    tmp_like += my_api.likes.getList(type='post',
                                                             owner_id='-' + group_id,
                                                             count = 1000,
                                                             item_id=post_id, 
                                                             offset = str(i))['items']
                except:
                    tmp_like = []
                time.sleep(1/(3 * len_api))
            
        
        else:
            my_api = next(iter_api)
            try:
                tmp_like = my_api.likes.getList(type='post',
                                                         owner_id='-' + group_id,
                                                         count = 1000,
                                                         item_id=post_id)['items']
            except:
                tmp_like = []

            time.sleep(1/(3 * len_api))
            
        likes_id[post_id] = tmp_like
    
    df_likes_user = pd.DataFrame([(like) for like in likes_id.items()], columns=['id', 'user_id_like'])
    df_likes_user.set_index('id', inplace=True)
    
    return df_likes_user

In [ ]:
def pars_likes_users_post_old(pars_wall, group_id):    
    likes_users_post = []
    likes_id = {}
    
    
    for post_id in tqdm_notebook(pars_wall['id']):
        likes_id[post_id] = my_api.likes.getList(type='post', 
                                                      owner_id='-' + group_id,
                                                      item_id=post_id)['items']

        time.sleep(0.4)
    
    df_likes_user = pd.DataFrame([(like) for like in likes_id.items()], columns=['id', 'user_id_like'])
    df_likes_user.set_index('id', inplace=True)
    
    return df_likes_user

In [ ]:
# def pars_likes_post_comment(comments_id, api_array, group_id):

#     likes_users_comment = []
    
#     iter_api = itertools.cycle(api_array)
#     len_api = len(api_array)
#     my_api = next(iter_api)
    
#     for comments_id, post_id in tqdm_notebook(zip([item['comment_id'] for item in comments_id],
#                                                  [item['post_id'] for item in comments_id])):
#         my_api = next(iter_api)
        
#         info = my_api.likes.getList(type='comment', 
#                                     owner_id='-' + group_id,
#                                     count = 1000,
#                                     item_id=comments_id)['items']

#         topic = [
#                 {            
#                     'like_comment_id': item,
#                     'post_id' : post_id
#                 } for item in info
#             ]
#         likes_users_comment.extend(topic)
#         time.sleep(1/(3 * len_api))

#     # здесь начинается кусок кода с болью, понять как это работает и как я к этому пришел, наверное,
#     # будет сложно, но очень хотелось сделать красиво, менее рукожопного способа я не нашел


#     df_tmp = pd.DataFrame(likes_users_comment).set_index('post_id') # пока все понятно
#     df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: str(x) + ' ') # делем строки из id
#     df_tmp = pd.DataFrame(df_tmp.groupby('post_id')['like_comment_id'].sum()) # группируем лайки по id поста
#     df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: (x.split(' '))) # делаем красиво
#     df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: x[:-1])

#     return df_tmp
    

In [ ]:
def pars_likes_post_comment(comments_id, api_array, group_id):

    likes_users_comment = []
    
    iter_api = itertools.cycle(api_array)
    len_api = len(api_array)
    my_api = next(iter_api)
    
    for comments_id, post_id in tqdm_notebook(zip([item['comment_id'] for item in comments_id],
                                                 [item['post_id'] for item in comments_id])):
        my_api = next(iter_api)
        try:
            n = my_api.likes.getList(type = 'comment',
                                     owner_id= '-' + group_id, 
                                     item_id=post_id)['count']
            tmp_like = []
            for i in range(0, n + 1000, 1000): 
                tmp_like += my_api.likes.getList(type='comment',
                                                             owner_id='-' + group_id,
                                                             count = 1000,
                                                             item_id=comments_id, 
                                                             offset = str(i))['items']
                time.sleep(1/(3 * len_api))
            
            
            
            
        except:
            
            tmp_like = []
            time.sleep(1/(3 * len_api))
            
        info = tmp_like
        
        
        topic = [
                {            
                    'like_comment_id': item,
                    'post_id' : post_id
                } for item in info
            ]
        likes_users_comment.extend(topic)
        time.sleep(1/(3 * len_api))

    # здесь начинается кусок кода с болью, понять как это работает и как я к этому пришел, наверное,
    # будет сложно, но очень хотелось сделать красиво, менее рукожопного способа я не нашел


    df_tmp = pd.DataFrame(likes_users_comment).set_index('post_id') # пока все понятно
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: str(x) + ' ') # делем строки из id
    df_tmp = pd.DataFrame(df_tmp.groupby('post_id')['like_comment_id'].sum()) # группируем лайки по id поста
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: (x.split(' '))) # делаем красиво
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: x[:-1])

    return df_tmp
    

In [ ]:
def pars_likes_post_comment_old(comments_id, group_id):

    likes_users_comment = []
    for comments_id, post_id in tqdm_notebook(zip([item['comment_id'] for item in comments_id],
                                                 [item['post_id'] for item in comments_id])):
        info = my_api.likes.getList(type='comment', 
                                    owner_id='-' + group_id, 
                                    item_id=comments_id)['items']

        topic = [
                {            
                    'like_comment_id': item,
                    'post_id' : post_id
                } for item in info
            ]
        likes_users_comment.extend(topic)
        time.sleep(1/3)

    # здесь начинается кусок кода с болью, понять как это работает и как я к этому пришел, наверное,
    # будет сложно, но очень хотелось сделать красиво, менее рукожопного способа я не нашел


    df_tmp = pd.DataFrame(likes_users_comment).set_index('post_id') # пока все понятно
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: str(x) + ' ') # делем строки из id
    df_tmp = pd.DataFrame(df_tmp.groupby('post_id')['like_comment_id'].sum()) # группируем лайки по id поста
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: (x.split(' '))) # делаем красиво
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: x[:-1])

    return df_tmp
    

In [ ]:
def pars_group_people(group_id, api_array): 
    # В этот словарик будем собирать информацию о текстах, можно напихать что-нибудь еще
    wall_inf = {'members' : [ ]}
    
    iter_api = itertools.cycle(api_array)
    len_api = len(api_array)
    my_api = next(iter_api)
    

    # Выясним сколько всего в группе подписчиков
    try:
        n = my_api.groups.getMembers(group_id = group_id)['count']
    except Exception:
        print('HiddenMembers?')
        wall_inf['members'].append('HiddenMembers')
        
    else:
        
        print('В группе ', n, 'подписчиков')

        for i in tqdm_notebook(range(0, n + 100, 100)): 
            # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
            my_api = next(iter_api)
            time.sleep(1/(3 * len_api))
            people = my_api.groups.getMembers(
                group_id = group_id, 
                count = 100, 
                offset = str(i)
            )['items']
            # Вытащили подписчиков 
            # Закинули всё это добро в словарик с данными 
            wall_inf['members'].extend(people)
    return wall_inf


In [ ]:
def parser(group_id, api_array):
    pars_wall = pars_group(group_id=group_id, api_array=api_array)
    df_post_user, comments_id = pars_comment(pars_wall = pars_wall, 
                                             api_array = api_array,
                                             group_id=group_id)
    df_like_user = pars_likes_users_post(pars_wall = pars_wall, 
                                         api_array=api_array,
                                         group_id=group_id)
    df_comm_like = pars_likes_post_comment(comments_id=comments_id,
                                           api_array = api_array,
                                           group_id=group_id)  
    df_pars = pd.DataFrame(pars_wall)
    df_pars = df_pars.join(df_post_user, on = 'id')
    df_pars = df_pars.join(df_like_user, on = 'id')
    df_pars = df_pars.join(df_comm_like, on = 'id')
    
    
    return df_pars 

In [ ]:
def save_df(df, name):
    with open('vk_df_' + str(name), 'wb') as f:
        pickle.dump(df, f)

In [ ]:
def open_df(name):
    with open(str(name), 'rb') as f:
        df = pickle.load(f)
    return df

In [ ]:
def fillna_with_arr(row):
    if row == 0:
        return []
    else:
        return row

In [ ]:
def plot_Like(df, type_plot):
    
    trace_like = go.Scatter(  
        x = df.index,
        y = df['like'],
        name = 'All Like'
    )

    trace_like_comment = go.Scatter(  
        x = df.index,
        y = df['like_comment'],
        name = 'Comment Like'
    )

    trace_like_post = go.Scatter(  
        x = df.index,
        y = df['like_post'],
        name = 'Post Like'
    )

    trace_like_com_com = go.Scatter(  
        x = df.index,
        y = df['like_comment_comment'],
        name = 'Comment of Post Like'
    )

    fig = make_subplots(rows=4, 
                        cols=1,
                        x_title='Number of post',
                        y_title='Like',
                        row_heights=[100, 100, 100, 100]
                       )

    fig.add_trace(trace_like_post, row=1, col=1)
    fig.add_trace(trace_like_comment, row=2, col=1)
    fig.add_trace(trace_like_com_com, row=3, col=1)
    fig.add_trace(trace_like, row=4, col=1)

    fig.update_layout(
        width = 950,
        height = 800
    )
    if type_plot == 'svg':
        fig.show('svg')
    else:
        fig.show()

In [ ]:
def plotMovingAverage_plotly(df_index, series, n, type_plot):

    rolling_mean = series.rolling(window=n).mean()
    rolling_std =  series.rolling(window=n).std()
    upper_bond = rolling_mean+1.96*rolling_std

    trace_upper_bond = go.Scatter(x = df_index, 
                                  y = upper_bond,
                                  name = 'Upper Bond',
                                  line = {'color': 'red', 'dash': 'dash', 'width': 2}
                                 )
    
    trace_mean = go.Scatter(x = df_index, 
                            y = rolling_std,
                            name = 'Moving average', 
                            line = {'color': 'green', 'dash': 'solid', 'width': 2}
                            )
    
    trace_actual = go.Scatter(x = df_index, 
                              y = series, 
                              name = 'Actual values',
                              line = {'color': 'blue', 'dash': 'solid', 'width': 1}
                             )

    fig_data = [trace_actual, trace_mean, trace_upper_bond]
    
    layout = go.Layout(height=500, 
                       width=950, 
                       title="Moving average window size = {}".format(n),
                       xaxis= dict(
                           title= 'Number of Post',
                           ticklen= 5,
                           zeroline= False,
                           gridwidth= 1
                       ),
                       yaxis=dict(
                           title= 'Like',
                           ticklen= 5,
                           gridwidth= 1
                       ))
    

    fig_fig = go.Figure(data = fig_data,
                        layout=layout)

    if type_plot == 'svg':
        fig_fig.show('svg')
    else:
        fig_fig.show()

In [ ]:
morph = pymorphy2.MorphAnalyzer()
noise = stopwords.words('russian') + list(punctuation)

# regex = re.compile("[А-Яа-я:=\)\()A-z\_\%/|]+")

regex = re.compile("[А-Яа-я]+")


def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

def lemmatize(text, pymorphy=morph):
    try:
        return " ".join([pymorphy.parse(word)[0].normal_form for word in text if word not in noise])
    except:
        return " " 
    
def clean_text(text):
    return lemmatize(words_only(text))


def clean_noise(text):
    return_arr = []
    for word in text.split():
        if word not in noise:
            return_arr.append(word)
    return return_arr


def clean_noise_arr(text):
    return_arr = []
    for word in text:
        if word not in noise:
            return_arr.append(word)
    return return_arr

def clean_df(df):    
    with Pool(8) as p:
        lemmas = list(tqdm_notebook(p.imap(clean_text, df['text']), total=len(df)))

    df['lemmas_text'] = lemmas

In [ ]:
def plot_reduce_viz(x, y, text, color_marker, title, type_plot):

    trace_ = go.Scatter(x = x, 
                        y = y,
                        mode='markers',
                        text = text,
                        marker={'color': color_marker}
                        )

    fig_data = [trace_]

    layout = go.Layout(height=500, 
                       width=950, 
                       title=title
                       )


    fig_fig = go.Figure(data = fig_data,
                        layout=layout)

    if type_plot == 'svg':
        fig_fig.show('svg')
    else:
        fig_fig.show()

In [ ]:
def clean_noise(text):
    return_arr = []
    for word in text.split():
        if word not in noise:
            return_arr.append(word)
    return return_arr

def count_word(text):    
    c_dict = Counter()
    for word in text:
        c_dict[word] += 1
    return c_dict

In [ ]:
def count_like(df, like_counter):    
    c_dict = Counter()
    for word in like_counter.keys():
        count_posts = 0
        for i in df[(df['target'] == 1)].index:
            if word in df['lemmas_text'][i]:
                c_dict[word] += df['like'][i]
                count_posts += 1
        c_dict[word] /= count_posts 
            
            
    return c_dict

In [ ]:
def rev_arr(arr):
    rev_arr = []
    for item in (reversed(arr)):
        rev_arr.append(item)
    return rev_arr

In [ ]:
def trace_count(counter, n_words): 
    x = []
    y = []
    for elements in (counter.most_common(n_words)):
        x.append(elements[0])
        y.append(elements[1])
    return x, y
    

In [ ]:
def plot_counter(x, y, title, type_plot):

    trace_ = go.Scatter(x = x, 
                        y = y,
                        name='word_like',
                        line = {'color': 'red'}
                        )

    fig_data = [trace_]

    layout = go.Layout(height=500, 
                       width=950, 
                       title=title
                       )


    fig_fig = go.Figure(data = fig_data,
                        layout=layout)

    if type_plot == 'svg':
        fig_fig.show('svg')
    else:
        fig_fig.show()

In [ ]:
def plot_line(x, y, title, type_plot):

    trace_ = go.Scatter(x = x, 
                        y = y,
                        name='word_like',
                        line = {'color': 'blue'}
                        )

    fig_data = [trace_]

    layout = go.Layout(height=500, 
                       width=950, 
                       title=title
                       )


    fig_fig = go.Figure(data = fig_data,
                        layout=layout)
    fig_fig.update_xaxes(rangeslider_visible=True)

    if type_plot == 'svg':
        fig_fig.show('svg')
    else:
        fig_fig.show()

In [ ]:
def plot_line_smooth(x, y, title, n, type_plot):
    rolling_mean = y.rolling(window=n).mean()
    trace_ = go.Scatter(x = x, 
                        y = rolling_mean,
                        name='word_like',
                        line = {'color': 'blue'}
                        )

    fig_data = [trace_]

    layout = go.Layout(height=500, 
                       width=950, 
                       title=title
                       )


    fig_fig = go.Figure(data = fig_data,
                        layout=layout)
    fig_fig.update_xaxes(rangeslider_visible=True)

    if type_plot == 'svg':
        fig_fig.show('svg')
    else:
        fig_fig.show()

In [ ]:
def count_like_top(df, arr):    
    c_dict = Counter()
    for word in arr:
        count_posts = 0
        for i in df['lemmas_text'].index:
            if word in df['lemmas_text'][i]:
                c_dict[word] += df['like'][i]
                count_posts += 1
        c_dict[word] /= count_posts 
            
            
    return c_dict

In [ ]:
def cloud_word(counter, n_words):
    plt.figure(figsize=(10, 10))

    wc = WordCloud(width=1600, height=1600, background_color="white", relative_scaling=0.5,
                   collocations=True, min_font_size=10).generate_from_frequencies(
        dict(counter.most_common(n_words)))
    plt.axis("off")
    plt.imshow(wc, interpolation="bilinear")
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.show()


In [ ]:
def count_words_in_df(x, word):
    if word in x.keys():
#         print('ok')
        return x[word]
    else:
        return 0

In [ ]:
def df_for_timeline(df, by, counter, n_words):
    df_tmp = df.copy()
    if by == 'date':
        df_tmp['date'] = df_tmp['date'].apply(lambda x: x.date())
#         df_tmp['date'] = df_tmp['date'].apply(lambda x: x.day)
    elif by == 'week':
        df_tmp['date'] = df_tmp['date'].apply(lambda x: x.week)
    elif by == 'month':
        df_tmp['date'] = df_tmp['date'].apply(lambda x: x.month)
    else:
        print('Input date, week or month.')
    df_date = pd.DataFrame(df_tmp.groupby(by = 'date')['counter'].sum())
    for word in counter.most_common(n_words):
#         print(word[0])
        df_date[word[0]] = df_date['counter'].apply(lambda x: count_words_in_df(x, word[0]))
    return df_date


In [ ]:
def color_gradient(x=0.0, start=(0, 0, 0), stop=(1, 1, 1)):
    r = np.interp(x, [0, 1], [start[0], stop[0]])
    g = np.interp(x, [0, 1], [start[1], stop[1]])
    b = np.interp(x, [0, 1], [start[2], stop[2]])
    return (r, g, b)

In [ ]:
def plot_word_dens(df, column, like_limit, date):

    plt.figure(dpi=380)
    fig, axes = joypy.joyplot(df,
                              column=columns_for_joy, 
                              overlap=2, 
                              ylim='own', 
                              x_range=(0, like_limit), 
                              fill=True, 
                              figsize=(10,12), 
                              legend=False, 
                              xlabels=True,
                              ylabels=True, 
                              colormap=lambda x: color_gradient(x, start=(.08, .45, .8)
                                                                 ,stop=(.8, .34, .44)), 
                              alpha=0.6, 
                              linewidth=.5, 
                              linecolor='w',
#                               background='k', # change to 'k' for black background or 'grey' for grey, 
                              fade=True
                             )
    plt.title('Плотности распределения лайков для топ-слов, агрегированных по {}'.format(date), 
              fontsize=14, 
              color='grey', 
              alpha=1)
    plt.rc("font", size=12)
    plt.xlabel('Лайки', fontsize=14, color='grey', alpha=1)
    plt.ylabel('Car Model', fontsize=8, color='grey', alpha=1)
    plt.show

In [ ]:
def plot_separate_line(df, counter, n_words, n_smooth, type_plot):
    df_tmp = df_for_timeline(df, 'date', counter, n_words)
    columns_for_joy = [column for column in df_tmp.columns if column not in 'counter']
    for column in columns_for_joy:
        plot_line_smooth(x = df_tmp.index, y = df_tmp[column], title = column, n = n_smooth, type_plot = type_plot)

In [ ]:
def plot_5_lines(df, counter, n_words, n_slice, n_smooth, type_plot):    
    df_tmp = df_for_timeline(df, 'date', counter, n_words)
    columns_for_joy = [column for column in df_tmp.columns if column not in 'counter']
    n_slice = n_slice
    
    
    rolling_mean_1 = df_tmp[df_tmp.columns[1+n_slice*5]].rolling(window=n_smooth).mean()
    rolling_mean_2 = df_tmp[df_tmp.columns[2+n_slice*5]].rolling(window=n_smooth).mean()
    rolling_mean_3 = df_tmp[df_tmp.columns[3+n_slice*5]].rolling(window=n_smooth).mean()
    rolling_mean_4 = df_tmp[df_tmp.columns[4+n_slice*5]].rolling(window=n_smooth).mean()
    rolling_mean_5 = df_tmp[df_tmp.columns[5+n_slice*5]].rolling(window=n_smooth).mean()
    
    trace_1 = go.Scatter(x = df_tmp.index, 
                        y = rolling_mean_1,
                        name=df_tmp.columns[1+n_slice*5],
                        line = {'color': 'blue'}
                        )

    trace_2 = go.Scatter(x = df_tmp.index, 
                        y = rolling_mean_2,
                        name=df_tmp.columns[2+n_slice*5],
                        line = {'color': 'red'}
                        )

    trace_3 = go.Scatter(x = df_tmp.index, 
                        y = rolling_mean_3,
                        name=df_tmp.columns[3+n_slice*5],
                        line = {'color': 'green'}
                        )

    trace_4 = go.Scatter(x = df_tmp.index, 
                        y = rolling_mean_4,
                        name=df_tmp.columns[4+n_slice*5],
                        line = {'color': 'orange'}
                        )

    trace_5 = go.Scatter(x = df_tmp.index, 
                        y = rolling_mean_5,
                        name=df_tmp.columns[5+n_slice*5],
                        line = {'color': 'purple'}
                        )

    fig_data = [trace_1, trace_2, trace_3, trace_4, trace_5]

    layout = go.Layout(height=500, 
                       width=950, 
                       title='Употребляемость слова',
                       )


    fig_fig = go.Figure(data = fig_data,
                        layout=layout)
    fig_fig.update_xaxes(rangeslider_visible=True)

    if type_plot == 'svg':
        fig_fig.show('svg')
    else:
        fig_fig.show()

In [ ]:
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'rb') as f:
        text = pickle.load(f)

    text = text.split()
    
    word_counts = count_word(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k:w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w:k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)
    
    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]


    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))

    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [ ]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [ ]:
def get_loss_and_train_op(net, lr=0.001):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)

        return criterion, optimizer

In [ ]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
        _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))